In [1]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)
from langchain.document_loaders import PyPDFLoader
import tiktoken

In [2]:
#pip install PyPDF

In [3]:
#pip install docx2txt -q

In [4]:
#pip install wikipedia -q

### CARGAR DOCUMENTOS

In [5]:
def load_document(file):
    import os
    name,extension = os.path.splitext(file)
    
    if extension == '.pdf':
        from langchain.document_loaders import PyPDFLoader
        print(f'Loading{file}')
        loader = PyPDFLoader(file)
    elif extension == '.docx':
        from langchain.document_loaders import Docx2txtLoader
        print(f'Loading {file}')
        loader = Docx2txtLoader(file)
    else:
        print("Documento no soportado")
        return None
    
    data = loader.load()
    return data

#Wikipedia
def load_from_wikipedia(query, lang = "en", load_max_docs = 2):
    from langchain.document_loaders import WikipediaLoader
    loader = WikipediaLoader(query=query, lang=lang,load_max_docs=load_max_docs)
    data = loader.load()
    return data

In [8]:
def chunk_data(data,chunk_size =256):
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=0)
    chunks = text_splitter.split_documents(data)
    return chunks

### CALCULAR EL COSTO DEL EMBEDDING

In [7]:
def print_embedding_cost(texts):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in texts])
    print(f'Total Tokens: {total_tokens}')
    print(f'Embedding Cost in USD: {total_tokens / 1000 * 0.0004:.6f}')

### EMBEDDING AND UPLOADING TO A VECTOR DATABASE (PINECONE)

In [9]:
def insert_or_fetch_embeddings(index_name):
    import pinecone
    from langchain.vectorstores import Pinecone
    from langchain.embeddings.openai import OpenAIEmbeddings
    
    embeddings = OpenAIEmbeddings()
    
    pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment = os.environ.get('PINECONE_ENV'))
    
    if index_name in pinecone.list_indexes():
                 print(f'Index {index_name} already exists. Loading embeddings ..', end = '')
                 vector_store = Pinecone.from_existing_index(index_name,embeddings)
                 print('Ok')
    else:
                 print(f'Creating index {index_name} and embeddings ...', end = '')
                 pinecone.create_index(index_name, dimension = 1536, metric = 'cosine')
                 vector_store = Pinecone.from_documents(chunks,embeddings, index_name = index_name)
                 print('Ok')
                 
    return vector_store

In [10]:
def delete_pinecone_index(index_name = 'all'):
    import pinecone
    pinecone.init(api_key = os.environ.get('PINECONE_API_KEY'), environment = os.environ.get('PINECONE_ENV'))
    if index_name == 'all':
                 indexes = pinecone.list_indexes()
                 print('Deleting all indexes ...')
                 for index in indexes:
                     pinecone.delete_index(index)
                 print('Ok')
    else:
                 print(f'Deleting index{index_name} ...', end = '')
                 pinecone.delete_index(index_name)
                 print('Ok')
    

### Asking and Getting Answers

In [11]:
def ask_and_get_answer(vector_store, q):
    from langchain.chains import RetrievalQA
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(model = 'gpt-3.5-turbo', temperature = 1)
    
    retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k': 3})
    
    chain = RetrievalQA.from_chain_type(llm = llm, chain_type = 'stuff', retriever = retriever)
    
    answer = chain.run(q)
    return answer

def ask_with_memory(vector_store, question, chat_history = []):
    from langchain.chains import ConversationalRetrievalChain
    from langchain.chat_models import ChatOpenAI
    
    llm = ChatOpenAI(temperature = 1)
    retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {'k': 3})
                                          
                                          
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    result = crc({'question':question, 'chat_history': chat_history})
    chat_history.append((question, result['answer']))
    
    return result, chat_history
                                

### Running Code

In [15]:
data = load_document('/Users/antonio.gutierrez/Documents/LLM_UDEMY/knowledge/M-SGI-002 Manual de Políticas del SGI_V10.pdf')
#print(data[10].page_content)
#print(data[10].metadata)
print(f'You have {len(data)} pages in your data')
print(f'There are {len(data)} characters in the page')

Loading/Users/antonio.gutierrez/Documents/LLM_UDEMY/knowledge/M-SGI-002 Manual de Políticas del SGI_V10.pdf
You have 54 pages in your data
There are 54 characters in the page


In [16]:
#data = load_document('/Users/anton/Documents/LLM_UDEMY/knowledge/la_divina_comedia.docx')
#print(data[0].page_content)

In [17]:
#data = load_from_wikipedia('GPT-4')
#print(data[0].page_content)

In [18]:
chunks = chunk_data(data)
print(len(chunks))
print(chunks[20].page_content)

571
Teletrabajo (Políticas Home Office).  ................................ ................................ ................................ .................  11


In [19]:
#print_embedding_cost(chunks)

In [20]:
#delete_pinecone_index()

In [18]:
index_name = 'askadocument'
vector_store = insert_or_fetch_embeddings(index_name)

C:\Users\anton\anaconda3\lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


Index askadocument already exists. Loading embeddings ..Ok


In [19]:
q = "de que trata el documento"
answer = ask_and_get_answer(vector_store, q)
print(answer)

El documento parece estar relacionado con la gestión de la seguridad de la información. Mencionan controles de acceso, retención de la información, documentación de procedimientos operacionales, gestión de cambios y gestión de la capacidad. También mencionan la recopilación inmediata de información impresa y el mantenimiento de escritorios libres de acceso directo a documentos. Sin embargo, sin más información, es difícil determinar el tema específico del documento.


In [ ]:
import time
i = 1
print('Escribe salir o gracias para Salir.')
while True:
    q = input(f'Pregunta#{1}: ')
    i = i+1
    if q.lower() in ['salir', 'gracias']:
        print('Ok ... ¡Nos vemos!')
        time.sleep(2)
        break
        
    answer = ask_and_get_answer(vector_store, q)
    print(f' \Respuesta: {answer}')
    print(f' \n {"-" * 50} \n')

In [21]:
#asking with memomory
chat_history = []
question = 'Cuantos temas toca el documento?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print (result['answer'])
print(chat_history)

El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.')]


In [23]:
question = 'Entonces solo toca dos temas el documento? ¿no toca otros temas?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.')]


In [24]:
question = 'multiplica los temas que toca por 2'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

El documento toca un total de 36 temas si los multiplicamos por 2.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.')]


In [26]:
question = '¿Entonces cuantos temas toca el documento?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

El documento parece abordar dos temas principales: el espacio físico adecuado para almacenar documentos en papel y los controles de seguridad de la norma ISO 27001:2013.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.'), ('¿Entonces cuantos temas toca el documento?', 'El documento parece abordar dos temas principales: el espacio físico adecuado para almacenar documentos en papel y los controles de seguridad de la norma ISO 27001:2013.')]


In [27]:
question = '¿y cuales son los temas secundarios?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

Los temas secundarios son:

1. Antecedentes no penales.
2. Estudio socioeconómico.
3. Estado de cuenta.
4. Número de seguro social.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.'), ('¿Entonces cuantos temas toca el documento?', 'El documento parece abordar dos temas principales: el espacio físico adecuado para almacenar documentos en papel y los controles de seguridad de la norma ISO 27001:2013.'), ('¿y cuales son los temas secundarios?', 'Los temas secundarios son:\n\n1. Antecedentes no penales.\n2. Estudio socioeconómico.\n3. Estad

In [28]:
question = '¿porqué dices que esos son los temas secundarios?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

Los temas secundarios son mencionados en el contexto como actividades o categorías de sitios a los que se debe tener acceso para llevar a cabo ciertas tareas en relación con la gestión de incidentes. Estos temas no son el enfoque principal del texto, pero se mencionan como parte de la labor que se debe realizar durante una investigación más profunda.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.'), ('¿Entonces cuantos temas toca el documento?', 'El documento parece abordar dos temas principales: el espacio físico adecuado para almace

In [29]:
question = 'relacionado al documento,¿Qué es la gestion de incidentes?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

Según el documento, la gestión de incidentes se refiere al proceso formal de reportar, responder y escalar los incidentes de seguridad de la información. Esto implica tener responsabilidades y procedimientos establecidos para manejar los incidentes, notificar los eventos de seguridad de la información y llevar a cabo sesiones de lecciones aprendidas con el personal involucrado en el manejo de incidentes.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.'), ('¿Entonces cuantos temas toca el documento?', 'El documento parece abordar dos te

In [30]:
question = '¿Cual es tu funcion?'
result, chat_history = ask_with_memory(vector_store, question, chat_history)
print(result['answer'])
print(chat_history)

No tengo una función específica, ya que soy un asistente virtual y mi propósito es proporcionar información y asistencia a los usuarios. No tengo la capacidad de desempeñar funciones o responsabilidades como un ser humano.
[('Cuantos temas toca el documento?', 'El documento menciona dos temas: almacenamiento físico de documentos en papel y control de versiones.'), ('Entonces solo toca dos temas el documento? ¿no toca otros temas?', 'El documento no menciona explícitamente otros temas relacionados con la organización interna en términos de concientización, educación y capacitación en seguridad de la información.'), ('multiplica los temas que toca por 2', 'El documento toca un total de 36 temas si los multiplicamos por 2.'), ('¿Entonces cuantos temas toca el documento?', 'El documento parece abordar dos temas principales: el espacio físico adecuado para almacenar documentos en papel y los controles de seguridad de la norma ISO 27001:2013.'), ('¿y cuales son los temas secundarios?', 'Los 